# Introduction to data analysis for natural and social sciences

## Exam first part

# Patient-specific Boolean models of signalling networks guide personalised treatments
Additional files at page 24.
Appendix at page 31.

## Prostate Boolean model construction
Results: 3

Appendix: 31

## Prostate Boolean model simulation
Results: 6

Appendix: 36

## Personalisation of the prostate Boolean model
Results: 6

Appendix: 37

## Personalised drug predictions of TCGA Boolean models
Results: 7

Appendix: 39

## Personalised drug predictions of LNCaP Boolean model
Results: 9

Appendix: 41

## Experimental validation of predicted targets
Results: 10

Appendix: 42

## Experimental validation of drugs in LNCaP
Results: 12

Appendix: 43

# Software

## Initial generic network extension
ROMA
OmniPath

GINsim

## Datasets
TGCA
GDSC

## Network visualization
Cytoscape

## Model simulation
MaBoSS